<a href="https://colab.research.google.com/github/cuilongjin/static/blob/main/colab-script/DeepFaceLab_Colab_V4_%E4%BD%BF%E7%94%A8%E5%85%B1%E4%BA%AB%E7%9B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，一路运行即可！

此脚本由**[托尼是塔克](https://wxp123.me)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

**为防止90分钟自动断开，请在浏览器控制台（按F12然后转到Console）中输入以下命令以防止断开。**
```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```
**这个脚本没有任何危害或影响，只是定时***调戏***以下这个页面以防止断开，如果复制的文件少就不用执行这一步。**

# 第一步 准备好workspace
这一步你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

In [ ]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录

%cd /content/drive/
!ls

%cd /content/drive/Shareddrives/wqdy/
!mkdir DeepFaceLab
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/


这一环节有两个选择：  
如果你第一次使用，可以加载示例项目，然后往后运行。  
如果你是二次运行，就不需要加载了。  
如果你希望上传本地素材，可以先用pack脚本把素材打包成faceset.pak到aligned目录，无须解压，可直接进行训练和转换，加载速度会很快。


In [ ]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

#第二步 安装DeepFaceLab
获取源代码，安装依赖,根据自己的情况选择版本  

last:最新版  

In [ ]:
#@title 重新安装

Version = "last" #@param ["v2.02.03", "v2.02.23b", "v2.02.28","v2.03.07","v2.03.15","v2.06.01","v2.06.19","last"]
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab
!rm -fr DeepFaceLab_Colab
!pip uninstall -y tensorflow
if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd ./DeepFaceLab_Colab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/iperov/DeepFaceLab.git  DeepFaceLab_Colab
  %cd ./DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd ./DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


In [ ]:
#@title 不重新安装，仅安装依赖

%cd /content/drive/Shareddrives/wqdy/DeepFaceLab
!pip uninstall -y tensorflow
%cd ./DeepFaceLab_Colab
!pip install -r requirements-colab.txt  
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# 第三步. 提取脸部

这个步骤其实是分两次的。第一次选src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。  
  
这一步，其实有三个小步骤，分解视频，提取人脸，打包素材。 如果有需要可以将他们拆分开来。一般是不用这么做的。 





In [ ]:
#@title 开始提取
target = "dst" #@param ["src","dst"]
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --output-debug
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


In [ ]:
#@title 开始排序
target = "dst" #@param ["src","dst"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned"
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned "

!python $cmd

# 第四步. 训练模型


*  支持SAEHD,Quick96等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [ ]:
#@title 开始训练
Model = "SAEHD" #@param ["SAEHD","Quick96"]

%cd "/content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

/content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab
Running trainer.

Choose one of saved models, or enter a name to create a new model.
[r] : rename
[d] : delete

[0] : zsy - latest
 : zsy1
zsy1

Model first run.

Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla T4

[0] Which GPU indexes to choose? : 
0


[0] Autobackup every N hour ( 0..24 ?:help ) : 0
[n] Write preview history ( y/n ?:help ) : 
n
[0] Target iteration : 
0
[y] Flip faces randomly ( y/n ?:help ) : 
y
[8] Batch_size ( ?:help ) : 
8
[128] Resolution ( 64-640 ?:help ) : 
128
[f] Face type ( h/mf/f/wf/head ?:help ) : 
f
[liae-ud] AE architecture ( ?:help ) : 
liae-ud
[256] AutoEncoder dimensions ( 32-1024 ?:help ) : 
256
[64] Encoder dimensions ( 16-256 ?:help ) : 
64
[64] Decoder dimensions ( 16-256 ?:help ) : 
64
[22] Decoder mask dimensions ( 16-256 ?:help ) : 
22
[n] Eyes and mouth priority ( y/n ?:help ) : 
n
[n] Uniform yaw distribution of samples ( y/n ?:help ) : 
n
[y] Pla

**模型预览方法请参考：**
https://www.wxp123.me/p/329

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

# 第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练H128,这里就选H128


In [ ]:
#@title 开始转换图片

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd "/content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py merge --output-mask-dir ../workspace/data_dst/merged_mask --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [ ]:
#@title 开始图片转视频
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged_mask --output-file ../workspace/result_mask.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio --lossless

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  


In [ ]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# 进入DeepFaceLab_Colab目录
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip uninstall -y tensorflow
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd 

# 工具



打包素材。可以加快上传，下载，加载的速度。

In [ ]:
#@title 素材打包

target = "src" #@param ["src","dst"]
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [ ]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [ ]:
#@title 素材增强

target = "src" #@param ["src"]
%cd /content/drive/Shareddrives/wqdy/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  

In [ ]:
#@title 移动到共享盘

target = "Shareddrives/wqdy" #@param ["Shareddrives/wqdy"]
%cd /content/drive/My Drive/DeepFaceLab/workspace
!cp -rfpv ./ /content/drive/Shareddrives/wqdy/DeepFaceLab/workspace

# 其他

  QQ群：659480116  
    
 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：https://www.wxp123.me
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!  
  
